# loss 的写法
不同项目的 loss 有不一样的书写条件 一般喜欢使用@staticmethod使得定义函数满足不用示例即可调用
一般loss的书写步骤包括
- 确定需要计算loss样本的label
- 计算loss
- 可以把accuracy/label/pred_label_id一同输出

## wechat_challenges cal_loss
- cal_loss 传入prediction和label 输出loss\accuracy\pred_label_id\label
- 计算accuracy时不反向传播 取label=pred_label_id pred_label_id使用prediction的最大值

## squeeze()
- 返回值为数组 并且不会修改原数组
- axis 用于指定需要删除的维度 但是指定的维度必须为单维度
- 若不指定维度 则会删除所有单维度的条目

In [1]:
import torch
import torch.nn.functional as F
import torch.nn as nn

In [ ]:
def cal_loss(prediction, label):
    label = label.squeeze(dim=1)
    loss = F.cross_entropy(prediction, label)
    # 计算loss时不反向传播
    with torch.no_grad():
        pred_label_id = torch.argmax(prediction, dim=1)
        accuracy = (label == pred_label_id).float().sum() / label.shape[0]
    return loss, accuracy, pred_label_id, label

## UPerNet part_loss
- part_loss 单纯用于算loss

In [ ]:
def part_loss(pred_part, gt_seg_part, gt_seg_object, object_label, valid):
    mask_object = (gt_seg_object == object_label)
    loss = F.nll_loss()
    loss= loss * mask_object.float()
    loss = torch.sum(loss.view(loss.size(0), -1), dim=1)
    nr_pixel = torch.sum(mask_object.view(mask_object.shape[0], -1), dim=1)
    sum_pixel = (nr_pixel * valid).sum()
    loss = (loss * valid.float()).sum() / torch.clamp(sum_pixel, 1).float()
    return loss

## wechat QQ multimodel learning get_pred_and_loss
- 这是一个多模态的案例 pred和loss在一起实现
- 包含了运行的代码 对于loss使用MSELoss(均方差损失函数)


In [ ]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
# set_random_seed(SEED)

def get_pred_and_loss(model, item, compute_loss=True, pairwise=True):
    video_feature = item['frame_features'].to(DEVICE)
    input_ids = item['id'].to(DEVICE)
    attention_mask = item['mask'].to(DEVICE)
    video_mask = item['frame_mask'].to(DEVICE)

    pred1, emb1, _ = model(video_feature, video_mask, input_ids, attention_mask, target=None, task=[])

    cos_sim = None
    loss = None
    target = None

    if pairwise:
        video_feature = item['frame_features2'].to(DEVICE)
        input_ids = item['id2'].to(DEVICE)
        attention_mask = item['mask2'].to(DEVICE)
        video_mask = item['frame_mask2'].to(DEVICE)

        pred2, emb2, _ = model(video_feature, video_mask, input_ids, attention_mask, target=None, task=[])
        cos_sim = torch.cosine_similarity(emb1, emb2)
        if compute_loss:
            target = item['target'].to(DEVICE)
            loss = nn.MSELoss()(cos_sim.view(-1), target.view(-1))
    return cos_sim, emb1, loss, target